In [1]:
import sys
sys.path.append('../')

import numpy as np
from numpy.typing import NDArray
import scipy
from dataclasses import dataclass
from typing import Callable
import matplotlib.pyplot as plt
import matplotlib as mpl
import itertools
import mpmath
import dill
import pickle
import os
import multiprocess as mp

from ray_delay.ray_detector import RayDetectorSpec, RayImpactSimulator
from stim_surface_code import patch, memory
from ray_delay.factory_simulator import Redundant15To1
from ray_delay.noise_model_patch import NoiseModelPatch
from ray_delay.noise_model import RayModelType, CosmicRayParams

import qc_utils.stats
import qc_utils.plot

mpmath.mp.dps = 50

In [2]:
dx = 7
dz = 3
dm = 3

f = Redundant15To1(dx, dz, dm, cache_cycles_per_distillation=True, mapping_mode='remap')
buffer_patch = memory.MemoryPatch(dx, dx, dm)
buffer_patch_nqubits = len(buffer_patch.all_qubits)
factory = Redundant15To1(7, 3, 3)
factory_nqubits = factory.num_phys_qubits

In [3]:
def expected_qubitcycle_overhead(ray_rate, recovery_time, online_chance, online_overhead):
    """Calculate the expected qubitcycle overhead for a given ray rate, recovery
    time, online chance, and online overhead. Assumes that more than one ray
    impact will always result in the factory being offline.
    
    Args:
        ray_rate: The rate of rays in rays/qubit/s.
        recovery_time: The time it takes to recover from a ray in seconds.
        online_chance: The chance that the factory is online given a ray impact.
        online_overhead: The expected qubitcycle overhead, given that a factory
            is online, during a ray impact.
    
    Returns:
        The expected qubitcycle overhead.
    """
    prob_no_rays = scipy.stats.poisson.pmf(0, factory_nqubits*ray_rate*recovery_time)
    prob_1_ray = scipy.stats.poisson.pmf(1, factory_nqubits*ray_rate*recovery_time)

    # distillations per time, relative to no rays
    ts_per_time_relative = prob_no_rays + prob_1_ray*online_chance/online_overhead
    
    return 1/ts_per_time_relative

In [4]:
def get_ideal_detector_spec(radius, strength):
    return RayDetectorSpec(
        detector_spatial_window_size=None,
        detector_temporal_window_size=None,
        ray_params=CosmicRayParams(min_radius=radius, max_radius=radius, max_strength=strength),
        ideal_detection=True,
    )

def calc_overhead_ideal_detection(factory, radius, strength):
    spec = get_ideal_detector_spec(radius, strength)
    return factory.calculate_avg_overhead_per_ray(spec, 1000, 1, 1e-7, save_cache=False)

In [5]:
def mpmath_poisson_ppf(p, lam, max_k=10):
    total_prob = 0
    k = 0
    while total_prob < p:
        total_prob += mpmath.exp(-lam)*mpmath.power(lam, mpmath.mpf(k))/mpmath.factorial(mpmath.mpf(k))
        k += 1
        if k > max_k:
            return 10**10
    return k-1

In [6]:
def distributed_overhead_ideal_detection(ray_rate, recovery_time):
    logical_err_rate = mpmath.mpf(1e-10)
    max_num_overlapping_rays = 1+mpmath_poisson_ppf(1-logical_err_rate, buffer_patch_nqubits*ray_rate*recovery_time)

    code_overheads = {0: 1, 1: 1, 2: 4, 3: 7, 4: 11, 5: 17}
    if max_num_overlapping_rays in code_overheads:
        return code_overheads[max_num_overlapping_rays]
    else:
        return 10**10

In [7]:
def expansion_overhead_ideal_detection(ray_rate, recovery_time, radius, strength, model='direct'):
    logical_err_rate = mpmath.mpf(1e-10)
    max_num_overlapping_rays = 1+mpmath_poisson_ppf(1-logical_err_rate, buffer_patch_nqubits*ray_rate*recovery_time)

    # chance_2_overlapping_rays = scipy.stats.poisson.pmf(2, buffer_patch_nqubits*ray_rate*recovery_time)

    if model == 'direct':
        extra_d = int(2*radius)
    else:
        assert model == 'scrambling'
        extra_d = int(radius)
    f_extra = Redundant15To1(dx+extra_d*max_num_overlapping_rays, dz+extra_d*max_num_overlapping_rays, dm)
    return f_extra.num_phys_qubits / factory_nqubits

In [8]:
generate_data = True

In [9]:
if generate_data:
    radii = np.linspace(2, 8, 10)
    strengths = 1-np.geomspace(0.001, 0.1, 12)[::-1]
    rates = np.geomspace(1e-7, 3e-3, 100)

    factory = Redundant15To1(7, 3, 3, cache_cycles_per_distillation=True, mapping_mode='remap', rng=0)
    assert len(factory._cycles_per_distillation_cache) > 0 # make sure cache loaded
    with mp.Pool(6) as pool:
        results = pool.starmap(calc_overhead_ideal_detection, [(factory, r, 1.0) for r in radii])
    remap_time_overheads = np.array([r[0] for r in results])
    remap_offline_rates = np.array([r[1] for r in results])

    net_overheads_offline = np.zeros((radii.shape[0], rates.shape[0]))
    net_overheads_remap = np.zeros((radii.shape[0], rates.shape[0]))
    for i,r in enumerate(radii):
            for j,rate in enumerate(rates):
                    net_overheads_remap[i,j] = expected_qubitcycle_overhead(rate, 1, 1-remap_offline_rates[i], remap_time_overheads[i])
                    net_overheads_offline[i,j] = expected_qubitcycle_overhead(rate, 1, 0, 1)

    net_overheads_distributed = np.zeros((radii.shape[0], rates.shape[0]))
    for j,rate in enumerate(rates):
            net_overheads_distributed[:,j] = distributed_overhead_ideal_detection(rate, 1)

    strengths = np.array([0.9, 0.99, 0.999])
    net_overheads_expansion_direct = np.zeros((radii.shape[0], rates.shape[0], strengths.shape[0]))
    net_overheads_expansion_scrambling = np.zeros((radii.shape[0], rates.shape[0]))

    for i,r in enumerate(radii):
        print(i, f'/ {radii.shape[0]}')
        with mp.Pool(6) as pool:
            for j,s in enumerate(strengths):
                net_overheads_expansion_direct[i,:,j] = pool.starmap(expansion_overhead_ideal_detection, [(rate, 1, r, s, 'direct') for rate in rates])
            net_overheads_expansion_scrambling[i,:] = pool.starmap(expansion_overhead_ideal_detection, [(rate, 1, r, 1, 'scrambling') for rate in rates])

    with open('data/evaluation_ideal_detection.pkl', 'wb') as f:
        dill.dump({
            'radii': radii,
            'rates': rates,
            'strengths': strengths,
            'overheads_remap': net_overheads_remap,
            'overheads_offline': net_overheads_offline,
            'overheads_distributed': net_overheads_distributed,
            'overheads_expansion_direct': net_overheads_expansion_direct,
            'overheads_expansion_scrambling': net_overheads_expansion_scrambling,
        }, f)
else:
    with open('data/evaluation_ideal_detection.pkl', 'rb') as f:
        data = dill.load(f)
        radii = data['radii']
        rates = data['rates']
        strengths = data['strengths']
        net_overheads_remap = data['overheads_remap']
        net_overheads_offline = data['overheads_offline']
        net_overheads_distributed = data['overheads_distributed']
        net_overheads_expansion_direct = data['overheads_expansion_direct']
        net_overheads_expansion_scrambling = data['overheads_expansion_scrambling']

0 / 10
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10


In [1]:
fig, ax = plt.subplots(1,4, figsize=(24, 8))

x, y = np.meshgrid(radii/2, rates, indexing='ij')

norm = mpl.colors.Normalize(vmin=1, vmax=32)

cmap = mpl.colors.LinearSegmentedColormap.from_list(
    'mycmap', 
    [
        (0, '#FFFFFF'),
        (norm(2), '#EEDD00'),
        (norm(6), '#DD9900'),
        (1, '#770000'),
    ]                                                    
)
cmap.set_over('#330000')
cmap.set_under('white')
cmap.set_bad('#330000')

ax[0].pcolormesh(x, y, net_overheads_distributed, norm=norm, cmap=cmap)
ax[0].set_title(r'$\it{Distributed}$ baseline')
ax[0].set_ylabel(r'$\Gamma \times T_{\text{offline}}$')

ax[1].pcolormesh(x, y, net_overheads_expansion_direct[:,:,1], norm=norm, cmap=cmap)
ax[1].set_title(r'$\it{Code}$ $\it{expansion}$ baseline' + '\n' + r'($\bf{Direct}$ noise model, $f_{T_1} = 0.01$)')

with plt.rc_context({'axes.edgecolor':'white', 'xtick.color':'white', 'ytick.color':'white', 'figure.facecolor':'black'}):
    ax_inset_1 = ax[1].inset_axes([0.2,0.6,0.3,0.3])
    ax_inset_1.pcolormesh(x, y, net_overheads_expansion_direct[:,:,0], norm=norm, cmap=cmap)
    ax_inset_1.loglog()
    ax_inset_1.tick_params(top=False, left=False, right=False, bottom=False, which='both')
    ax_inset_1.set_yticklabels([])
    ax_inset_1.xaxis.set_major_formatter(mpl.ticker.NullFormatter())
    ax_inset_1.xaxis.set_minor_formatter(mpl.ticker.NullFormatter())
    ax_inset_1.set_title(r'$f_{T_1} = 0.1$', fontsize=10, color='white')

    ax_inset_2 = ax[1].inset_axes([0.6,0.6,0.3,0.3])
    ax_inset_2.pcolormesh(x, y, net_overheads_expansion_direct[:,:,2], norm=norm, cmap=cmap)
    ax_inset_2.loglog()
    ax_inset_2.tick_params(top=False, left=False, right=False, bottom=False, which='both')
    ax_inset_2.set_yticklabels([])
    ax_inset_2.xaxis.set_major_formatter(mpl.ticker.NullFormatter())
    ax_inset_2.xaxis.set_minor_formatter(mpl.ticker.NullFormatter())
    ax_inset_2.set_title(r'$f_{T_1} = 0.001$', fontsize=10, color='white')

pcolormesh = ax[2].pcolormesh(x, y, net_overheads_expansion_scrambling, norm=norm, cmap=cmap)
ax[2].set_title(r'$\it{Code}$ $\it{expansion}$ baseline' + '\n' + r'($\bf{Scrambling}$ noise model)')

ax[3].pcolormesh(x, y, net_overheads_remap, norm=norm, cmap=cmap)
ax[3].set_title('Re-mapping (ours)')
ax[3].plot([3], [1/27/10*50e-3], 'x', color='k')
ax[3].annotate('McEwen 2022', (3, 1/27/10*50e-3), textcoords='offset points', xytext=(5,5), ha='left', va='center', color='k')
ax[3].plot([2.5], [1/27/16.4*1], 'x', color='k')
ax[3].annotate('Thorbeck 2023', (2.5, 1/27/16.4*1), textcoords='offset points', xytext=(5,5), ha='left', va='center', color='k')
ax[3].plot([1], [1/4/50*50e-3], 'x', color='k')
ax[3].annotate('Wilen 2021', (1, 1/4/50*50e-3), textcoords='offset points', xytext=(5,5), ha='left', va='center', color='k')

for i,axis in enumerate(ax):
    axis.set_yscale('log')
    # if i == 2:
    #     axis.set_xticks([])
    # else:
    axis.set_xlabel(r'Ray radius $r_{\text{CRE}}$')
    axis.set_xticks([1,2,3,4])

cbar = qc_utils.plot.add_cbar(ax[3], norm, cmap, size=0.01)
# cbar = plt.colorbar(pcolormesh, ticks=[1, net_overheads_offline.max(), 4, 11,
# 17])
cbar.set_ticks([1,2,4,7,11,17])
cbar.set_label('Relative qubitcycle cost per distillation')

plt.savefig('../figures/ideal_detection_overheads.svg', bbox_inches='tight')
plt.show()

NameError: name 'plt' is not defined

In [90]:
def geo_mean(iterable):
    a = np.array(iterable).flatten()
    return a.prod()**(1.0/len(a))

best_baseline_direct = np.minimum(net_overheads_distributed, net_overheads_expansion_direct[:,:,0])
best_baseline_improvements_direct = (best_baseline_direct-1) / (net_overheads_remap-1)
best_baseline_improvements_direct = best_baseline_improvements_direct[np.isfinite(best_baseline_improvements_direct)]
best_baseline_scrambling = np.minimum(net_overheads_distributed, net_overheads_expansion_scrambling)
best_baseline_improvements_scrambling = (best_baseline_direct-1) / (net_overheads_remap-1)
best_baseline_improvements_scrambling = best_baseline_improvements_scrambling[np.isfinite(best_baseline_improvements_scrambling)]

print('Min improvement over best other method (direct):', best_baseline_improvements_direct.min())
print('Min improvement over best other method (scrambling):', best_baseline_improvements_scrambling.min())

print('Max improvement over best other method (direct):', best_baseline_improvements_direct.max())
print('Max improvement over best other method (scrambling):', best_baseline_improvements_scrambling.max())

print('Geomean improvement over best other method (direct):', geo_mean([mpmath.mpf(x) for x in best_baseline_improvements_direct]))
print('Geomean improvement over best other method (scrambling):', geo_mean([mpmath.mpf(x) for x in best_baseline_improvements_scrambling]))

best_baseline_qubitcycle_improvements_direct = best_baseline_direct / net_overheads_remap
best_baseline_qubitcycle_improvements_direct = best_baseline_qubitcycle_improvements_direct[np.isfinite(best_baseline_qubitcycle_improvements_direct)]
best_baseline_qubitcycle_improvements_scrambling = best_baseline_direct / net_overheads_remap
best_baseline_qubitcycle_improvements_scrambling = best_baseline_qubitcycle_improvements_scrambling[np.isfinite(best_baseline_qubitcycle_improvements_scrambling)]

print('Min qubitcycle improvement over best other method (direct):', best_baseline_qubitcycle_improvements_direct.min())
print('Min qubitcycle improvement over best other method (scrambling):', best_baseline_qubitcycle_improvements_scrambling.min())

print('Max qubitcycle improvement over best other method (direct):', best_baseline_qubitcycle_improvements_direct.max())
print('Max qubitcycle improvement over best other method (scrambling):', best_baseline_qubitcycle_improvements_scrambling.max())

print('Geomean qubitcycle improvement over best other method (direct):', geo_mean([mpmath.mpf(x) for x in best_baseline_qubitcycle_improvements_direct]))
print('Geomean qubitcycle improvement over best other method (scrambling):', geo_mean([mpmath.mpf(x) for x in best_baseline_qubitcycle_improvements_scrambling]))

Min improvement over best other method (direct): 6.044950800666828
Min improvement over best other method (scrambling): 6.044950800666828
Max improvement over best other method (direct): 75641.08541172852
Max improvement over best other method (scrambling): 75641.08541172852
Geomean improvement over best other method (direct): 1276.2176545911981463812973542563126579565919284554
Geomean improvement over best other method (scrambling): 1276.2176545911981463812973542563126579565919284554
Min qubitcycle improvement over best other method (direct): 3.999625816644771
Min qubitcycle improvement over best other method (scrambling): 3.999625816644771
Max qubitcycle improvement over best other method (direct): 244.26533314963234
Max qubitcycle improvement over best other method (scrambling): 244.26533314963234
Geomean qubitcycle improvement over best other method (direct): 13.692556528370526598363610013707528083892894957756
Geomean qubitcycle improvement over best other method (scrambling): 13.6

In [51]:
scipy.stats.poisson.pmf(2, 1/27/10*50e-3)

1.7143601371065397e-08

# Plotting $\Gamma$ and $T_{\text{offline}}$ for our  method

# Finding max and average qubitcycle improvement numbers

# Bar plot for specific parameter settings (e.g. Google Direct, Scrambling (fast recal), Scrambling (slow recal), ...|)

In [94]:
scipy.stats.poisson.pmf(1, 25e-3)

0.02438274780070832